In [2]:
import numpy as np
import sklearn
import math
import os
import cv2
import pickle as pkl

In [5]:
dir_list = sorted(os.listdir("./output/"))

In [11]:
def read_file(path):
    with open(path, 'rb') as f:
        result = pkl.load(f)
    return result

In [12]:
output_list = [read_file('./output/' + file_name) for file_name in dir_list]

In [13]:
output_list[0]

[[591.75, 377.8, 752.8, 541.0, 1, 1],
 [556.65, 119.98, 604.85, 164.04000000000002, 1, 1],
 [544.2, 87.27, 580.45, 118.35, 1, 1],
 [507.35, 66.5, 536.35, 93.425, 1, 1],
 [552.0, 69.095, 582.55, 89.78999999999999, 1, 1],
 [730.1, 113.23, 783.5, 154.18, 1, 1],
 [901.15, 249.12, 961.0, 358.11, 1, 1]]

In [14]:
all_box_value = []
for boxes in output_list:
    for box in boxes:
        width = box[2]-box[0]
        height = box[3] - box[1]
        all_box_value.append([width,height])

In [30]:
import numpy as np


class YOLO_Kmeans:

    def __init__(self, cluster_number, boxes):
        self.cluster_number = cluster_number
        self.boxes = boxes

    def iou(self, boxes, clusters):  # 1 box -> k clusters
        n = boxes.shape[0]
        k = self.cluster_number

        box_area = boxes[:, 0] * boxes[:, 1]
        box_area = box_area.repeat(k)
        box_area = np.reshape(box_area, (n, k))

        cluster_area = clusters[:, 0] * clusters[:, 1]
        cluster_area = np.tile(cluster_area, [1, n])
        cluster_area = np.reshape(cluster_area, (n, k))

        box_w_matrix = np.reshape(boxes[:, 0].repeat(k), (n, k))
        cluster_w_matrix = np.reshape(np.tile(clusters[:, 0], (1, n)), (n, k))
        min_w_matrix = np.minimum(cluster_w_matrix, box_w_matrix)

        box_h_matrix = np.reshape(boxes[:, 1].repeat(k), (n, k))
        cluster_h_matrix = np.reshape(np.tile(clusters[:, 1], (1, n)), (n, k))
        min_h_matrix = np.minimum(cluster_h_matrix, box_h_matrix)
        inter_area = np.multiply(min_w_matrix, min_h_matrix)

        result = inter_area / (box_area + cluster_area - inter_area)
        return result

    def avg_iou(self, boxes, clusters):
        accuracy = np.mean([np.max(self.iou(boxes, clusters), axis=1)])
        return accuracy

    def kmeans(self, boxes, k, dist=np.median):
        box_number = boxes.shape[0]
        distances = np.empty((box_number, k))
        last_nearest = np.zeros((box_number,))
        np.random.seed()
        clusters = boxes[np.random.choice(
            box_number, k, replace=False)]  # init k clusters
        while True:

            distances = 1 - self.iou(boxes, clusters)

            current_nearest = np.argmin(distances, axis=1)
            if (last_nearest == current_nearest).all():
                break  # clusters won't change
            for cluster in range(k):
                clusters[cluster] = dist(  # update clusters
                    boxes[current_nearest == cluster], axis=0)

            last_nearest = current_nearest

        return clusters

    def result2txt(self, data):
        f = open("yolo_anchors.txt", 'w')
        row = np.shape(data)[0]
        for i in range(row):
            if i == 0:
                x_y = "%d,%d" % (data[i][0], data[i][1])
            else:
                x_y = ", %d,%d" % (data[i][0], data[i][1])
            f.write(x_y)
        f.close()

    def txt2boxes(self):
        result = []
        for boxes in self.boxes:
            for box in boxes:
                width = box[2]-box[0]
                height = box[3]-box[1]
                result.append([width, height])
        result = np.array(result)
        return result

    def txt2clusters(self):
        all_boxes = self.txt2boxes()
        result = self.kmeans(all_boxes, k=self.cluster_number)
        result = result[np.lexsort(result.T[0, None])]
        self.result2txt(result)
        print("K anchors:\n {}".format(result))
        print("Accuracy: {:.2f}%".format(
            self.avg_iou(all_boxes, result) * 100))

In [31]:
cluster_number = 9
kmeans = YOLO_Kmeans(cluster_number,output_list)

In [32]:
kmeans.txt2clusters()

K anchors:
 [[ 28.66   23.55 ]
 [ 39.33   31.863]
 [ 54.27   39.55 ]
 [ 66.64   56.403]
 [ 92.67   85.2  ]
 [ 93.73   48.72 ]
 [131.07   62.8  ]
 [159.87  105.68 ]
 [236.88  165.68 ]]
Accuracy: 78.24%


In [33]:
iou(all_box_value, 9)

TypeError: 'int' object is not subscriptable